In [ ]:
import json
import requests
import openai
from openai.types.chat import ChatCompletionMessage

movie_url = "https://nomad-movies.nomadcoders.workers.dev"
client = openai.OpenAI()
messages = []


def get_popular_movies():
    try:
        response = requests.get(movie_url + "/movies")
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        return f"Error fetching movies: {e}"


def get_movie_details(id):
    try:
        response = requests.get(f"{movie_url}/movies/{id}")
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        return f"Error fetching movie details: {e}"


def get_similar_movies(id):
    try:
        response = requests.get(f"{movie_url}/movies/{id}/similar")
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        return f"Error fetching movie details: {e}"


FUNCTION_MAP = {
    "get_popular_movies": get_popular_movies,
    "get_movie_details": get_movie_details,
    "get_similar_movies": get_similar_movies,
}


TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_popular_movies",
            "description": "Get a list of popular movies and their ids. Use this first when you only know the movie title and need to look up its id.",
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_movie_details",
            "description": "Get detailed information about a movie by id. The id must come from a previous tool call such as get_popular_movies; never make up an id.",
            "parameters": {
                "type": "object",
                "properties": {
                    "id": {
                        "type": "string",
                        "description": "The id of the movie, taken from the results of get_popular_movies or another tool output.",
                    }
                },
                "required": ["id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_similar_movies",
            "description": "Get similar movies given a movie id. The id must come from a previous tool call such as get_popular_movies or get_movie_details; never make up an id.",
            "parameters": {
                "type": "object",
                "properties": {
                    "id": {
                        "type": "string",
                        "description": "The id of the movie, taken from the results of get_popular_movies or another tool output.",
                    }
                },
                "required": ["id"],
            },
        },
    },
]


def process_ai_response(message: ChatCompletionMessage):
    if message.tool_calls:
        messages.append(
            {
                "role": "assistant",
                "content": message.content or "",
                "tool_calls": [
                    {
                        "id": tool_call.id,
                        "type": "function",
                        "function": {
                            "name": tool_call.function.name,
                            "arguments": tool_call.function.arguments,
                        },
                    }
                    for tool_call in message.tool_calls
                ],
            }
        )

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments
            print(f"calling Function: {function_name} with arguments: {arguments}")

            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}

            function_to_run = FUNCTION_MAP.get(function_name)
            result = function_to_run(**arguments)

            if isinstance(result, str):
                content = result
            else:
                content = json.dumps(result)

            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": content,
                }
            )

        call_ai()
    else:
        messages.append({"role": "assistant", "content": message.content})


def call_ai():
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=TOOLS,
    )
    message = response.choices[0].message

    # Print in the requested format
    if message.tool_calls:
        tool_names = ", ".join(
            f"{tool_call.function.name}()" for tool_call in message.tool_calls
        )
        print(f"Agent: [{tool_names} called]")

    print(f"Agent: {message.content}")
    process_ai_response(message)


while True:
    message = input("Send a message to llm")
    if message == "quit":
        break
    else:
        messages.append({"role": "user", "content": message})
        print(f"User: {message}")
        call_ai()
